# Improved SCA based on SVM
---

In this notebook, we will train a model for each bit of the intermediate value to predict its value, targeting a specific byte of the encryption key similarly to `10_train_HW_model`

---

#### 1. Importing training traces

In [61]:
import chipwhisperer as cw
import numpy as np

In [62]:
# To use the included Traces
# project_file = 'SCA_SVM_DATA/Included/traces/training_traces.cwp'

project_folder = 'SCA_SVM_DATA'    
traces_folder = f'{project_folder}/traces'
project_file = f'{traces_folder}/training_traces.cwp'

project = cw.open_project(project_file)
traces = np.array(project.waves)
plaintexts = np.array(project.textins)
key = project.keys[0]
hex_key = ''.join(f'{byte:02x}' for byte in key)
print(f"✅ Loaded {len(traces)} traces and textins with key {hex_key}")

✅ Loaded 20000 traces and textins with key 2b7e151628aed2a6abf7158809cf4f3c


In [63]:
SBOX = np.array([
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16
])

In [64]:
# Get the bit value at a given position (0-7)
def get_bit(value, bit_pos):
    return (value >> bit_pos) & 1

#### 2. Feature Selection
Each bit may have different samples that correlate best with the intermediate value, so we will select the most relevant samples for each bit and save the selector.

In [65]:
# Compute intermediate values of a specific bit.
def calculate_intermediates(plaintexts, byte_index=0, bit_index=0):
    
    pt_bytes = plaintexts[:,byte_index]
    sbox_outputs = np.array([SBOX[pt ^ KEY[byte_index]] for pt in pt_bytes])
    
    return np.array([get_bit(s, bit_index) for s in sbox_outputs])

In [66]:
from scipy.stats import pearsonr

# Compute Pearson correlation for each trace point.
def pearson_corr(X, y):
    return np.array([ abs(pearsonr(X[:, i], y)[0]) for i in range(X.shape[1])])

In [67]:
from sklearn.feature_selection import SelectKBest

# Feature selection using Pearson correlation.
def perform_feature_selection(traces, plaintexts, byte_index=0, bit_index=0, num_features=600):
    
    intermediates = calculate_intermediates(plaintexts, byte_index, bit_index)
    
    selector = SelectKBest(pearson_corr, k=num_features)
    selector.fit(traces, intermediates)
    
    correlations = pearson_corr(traces, intermediates)
    selected_points = selector.get_support(indices=True)
    
    print(f"Feature selection for byte {byte_index}, bit {bit_index}:")
    print(f"- Selected {len(selected_points)} points from {traces.shape[1]} total")
    print(f"- Max correlation: {np.max(correlations):.4f}")
    
    return selector, intermediates, correlations

In [68]:
import warnings
from scipy.stats import ConstantInputWarning

# To suppress unnecessary warnings
warnings.filterwarnings("ignore", category=ConstantInputWarning)

In [69]:
import os

target_byte = 0 

selectors_data = []
for bit in range(8):
    print(f"\n--- Feature selection for bit {bit} ---")
    bit_selector, bit_intermediates, bit_correlations = perform_feature_selection(
        traces, plaintexts, byte_index=target_byte, bit_index=bit, num_features=600 )
    
    selector = {'selector': bit_selector, 'intermediates': bit_intermediates}
    selectors_data.append(selector)


--- Feature selection for bit 0 ---
Feature selection for byte 0, bit 0:
- Selected 600 points from 5000 total
- Max correlation: 0.2837

--- Feature selection for bit 1 ---
Feature selection for byte 0, bit 1:
- Selected 600 points from 5000 total
- Max correlation: 0.3483

--- Feature selection for bit 2 ---
Feature selection for byte 0, bit 2:
- Selected 600 points from 5000 total
- Max correlation: 0.2907

--- Feature selection for bit 3 ---
Feature selection for byte 0, bit 3:
- Selected 600 points from 5000 total
- Max correlation: 0.3498

--- Feature selection for bit 4 ---
Feature selection for byte 0, bit 4:
- Selected 600 points from 5000 total
- Max correlation: 0.3265

--- Feature selection for bit 5 ---
Feature selection for byte 0, bit 5:
- Selected 600 points from 5000 total
- Max correlation: 0.3184

--- Feature selection for bit 6 ---
Feature selection for byte 0, bit 6:
- Selected 600 points from 5000 total
- Max correlation: 0.2885

--- Feature selection for bit 7 -

#### 3. Training model for each bit

In [70]:
from sklearn.feature_selection import f_regression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

In [71]:
def prepare_X_y(traces, selector, intermediates, M, N):
    X = selector.transform(traces)[:M,:N]
    y = intermediates[:M]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test

In [72]:
def optimize_and_train_model(traces, selector, intermediates, M_values, N_values):
    best_model_data = {}
    best_score = 0
    
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [0.1, 1, 2],
        'kernel': ['rbf', 'linear', 'polynomial', 'sigmoid'] 
    }
    
    for M in M_values:
        for N in N_values:
            X_train, X_test, y_train, y_test = prepare_X_y(traces, selector, intermediates, M, N)
            
            # Perform grid search
            svm = SVC(decision_function_shape='ovo')
            grid_search = GridSearchCV(svm, param_grid, cv=2, n_jobs=-1, verbose=1)
            grid_search.fit(X_train, y_train)
            
            model = grid_search.best_estimator_
            y_pred = model.predict(X_test)
            score = accuracy_score(y_test, y_pred)
            
            # Save the best estimator
            if score > best_score:
                best_model_data = {'model': model, 'M': M, 'N': N, 'params': grid_search.best_params_, 'score': score}
                
    # Print result
    print(f"Best estimator with M={best_model_data['M']}, N={best_model_data['N']}, Params:{best_model_data['params']} -> Score:{best_model_data['score']}")    
    
    return best_model_data

In [73]:
M_values = [x for x in range(2000, 20001, 2000)]
N_values = [x for x in range(50, 601, 50)]

In [74]:
models_folder = f'{project_folder}/models'
os.makedirs(models_folder, exist_ok = True)

In [76]:
import joblib

for bit, selector_data in enumerate(selectors_data):
    print(f'\n--- Training bit{bit} ---')
    selector = selector_data['selector']
    intermediates = selector_data['intermediates']
    model_data = optimize_and_train_model(traces, selector, intermediates, M_values, N_values)
    model_data['selector'] = selector
    
    model_file = f'{models_folder}/bit{bit}_model.pkl'
    joblib.dump(model_data, model_file)
    print(f"Model saved to {model_file}")


--- Training bit0 ---
Best estimator with M=4000, N=250, Params:{'C': 100, 'gamma': 1, 'kernel': 'rbf'} -> Score:0.6819
Model saved to SCA_SVM_DATA/models/bit0_model.pkl

--- Training bit1 ---
Best estimator with M=10000, N=300, Params:{'C': 100, 'gamma': 1, 'kernel': 'rbf'} -> Score:0.7375
Model saved to SCA_SVM_DATA/models/bit1_model.pkl

--- Training bit2 ---
Best estimator with M=8000, N=250, Params:{'C': 100, 'gamma': 1, 'kernel': 'rbf'} -> Score:0.7411
Model saved to SCA_SVM_DATA/models/bit2_model.pkl

--- Training bit3 ---
Best estimator with M=10000, N=300, Params:{'C': 100, 'gamma': 1, 'kernel': 'rbf'} -> Score:0.7981
Model saved to SCA_SVM_DATA/models/bit3_model.pkl

--- Training bit4 ---
Best estimator with M=8000, N=250, Params:{'C': 100, 'gamma': 1, 'kernel': 'rbf'} -> Score:0.7678
Model saved to SCA_SVM_DATA/models/bit4_model.pkl

--- Training bit5 ---
Best estimator with M=10000, N=250, Params:{'C': 100, 'gamma': 1, 'kernel': 'rbf'} -> Score:0.825
Model saved to SCA_SVM